In [1]:
# import
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, func
import psycopg2
from config import pwd

In [23]:
hate_crimeDF = pd.read_csv('hate_crime.csv')
hate_crimeDF.head()

,INCIDENT_ID,DATA_YEAR,ORI,PUB_AGENCY_NAME,PUB_AGENCY_UNIT,AGENCY_TYPE_NAME,STATE_ABBR,STATE_NAME,DIVISION_NAME,REGION_NAME,...,OFFENDER_RACE,OFFENDER_ETHNICITY,VICTIM_COUNT,OFFENSE_NAME,TOTAL_INDIVIDUAL_VICTIMS,LOCATION_NAME,BIAS_DESC,VICTIM_TYPES,MULTIPLE_OFFENSE,MULTIPLE_BIAS
0,3015,1991,AR0040200,Rogers,NaN,City,AR,Arkansas,West South Central,South,...,White,NaN,1,Intimidation,1.0,Highway/Road/Alley/Street/Sidewalk,Anti-Black or African American,Individual,S,S
1,3016,1991,AR0290100,Hope,NaN,City,AR,Arkansas,West South Central,South,...,Black or African American,NaN,1,Simple Assault,1.0,Highway/Road/Alley/Street/Sidewalk,Anti-White,Individual,S,S
2,43,1991,AR0350100,Pine Bluff,NaN,City,AR,Arkansas,West South Central,South,...,Black or African American,NaN,1,Aggravated Assault,1.0,Residence/Home,Anti-Black or African American,Individual,S,S
3,44,1991,AR0350100,Pine Bluff,NaN,City,AR,Arkansas,West South Central,South,...,Black or African American,NaN,2,Aggravated Assault;Destruction/Damage/Vandalis...,1.0,Highway/Road/Alley/Street/Sidewalk,Anti-White,Individual,M,S
4,3017,1991,AR0350100,Pine Bluff,NaN,City,AR,Arkansas,West South Central,South,...,Black or African American,NaN,1,Aggravated Assault,1.0,Service/Gas Station,Anti-White,Individual,S,S


In [24]:
#dropping unnecessary rows
to_drop = ['ORI',
            'PUB_AGENCY_UNIT',
            'DIVISION_NAME',
            'OFFENDER_RACE',
            'OFFENDER_ETHNICITY',
            'VICTIM_COUNT',
            'OFFENSE_NAME',
            'LOCATION_NAME',
            'VICTIM_TYPES',
            'MULTIPLE_OFFENSE',
            'MULTIPLE_BIAS',
            'POPULATION_GROUP_CODE',
            'POPULATION_GROUP_DESC',
            'ADULT_VICTIM_COUNT',
            'JUVENILE_VICTIM_COUNT',
            'TOTAL_OFFENDER_COUNT',
            'ADULT_OFFENDER_COUNT',
            'JUVENILE_OFFENDER_COUNT']
hate_crimeDF.drop(to_drop, inplace=True, axis=1)
hate_crimeDF.head()

,INCIDENT_ID,DATA_YEAR,PUB_AGENCY_NAME,AGENCY_TYPE_NAME,STATE_ABBR,STATE_NAME,REGION_NAME,INCIDENT_DATE,TOTAL_INDIVIDUAL_VICTIMS,BIAS_DESC
0,3015,1991,Rogers,City,AR,Arkansas,South,31-AUG-91,1.0,Anti-Black or African American
1,3016,1991,Hope,City,AR,Arkansas,South,19-SEP-91,1.0,Anti-White
2,43,1991,Pine Bluff,City,AR,Arkansas,South,04-JUL-91,1.0,Anti-Black or African American
3,44,1991,Pine Bluff,City,AR,Arkansas,South,24-DEC-91,1.0,Anti-White
4,3017,1991,Pine Bluff,City,AR,Arkansas,South,23-DEC-91,1.0,Anti-White


In [25]:
# Checking the data types of all columns
hate_crimeDF.dtypes

INCIDENT_ID                   int64
DATA_YEAR                     int64
PUB_AGENCY_NAME              object
AGENCY_TYPE_NAME             object
STATE_ABBR                   object
STATE_NAME                   object
REGION_NAME                  object
INCIDENT_DATE                object
TOTAL_INDIVIDUAL_VICTIMS    float64
BIAS_DESC                    object
dtype: object

In [26]:
# Checking the whether the the row contains null values
hate_crimeDF.isna().sum()

INCIDENT_ID                    0
DATA_YEAR                      0
PUB_AGENCY_NAME                0
AGENCY_TYPE_NAME               0
STATE_ABBR                     0
STATE_NAME                     0
REGION_NAME                    0
INCIDENT_DATE                  0
TOTAL_INDIVIDUAL_VICTIMS    1606
BIAS_DESC                      0
dtype: int64

In [27]:
#removing null values
hate_crimeDF.dropna(axis=0, how='any', thresh=None, subset=['TOTAL_INDIVIDUAL_VICTIMS'], inplace=True)
hate_crimeDF.isna().sum()

INCIDENT_ID                 0
DATA_YEAR                   0
PUB_AGENCY_NAME             0
AGENCY_TYPE_NAME            0
STATE_ABBR                  0
STATE_NAME                  0
REGION_NAME                 0
INCIDENT_DATE               0
TOTAL_INDIVIDUAL_VICTIMS    0
BIAS_DESC                   0
dtype: int64

In [28]:
hate_crimeDF.head()

,INCIDENT_ID,DATA_YEAR,PUB_AGENCY_NAME,AGENCY_TYPE_NAME,STATE_ABBR,STATE_NAME,REGION_NAME,INCIDENT_DATE,TOTAL_INDIVIDUAL_VICTIMS,BIAS_DESC
0,3015,1991,Rogers,City,AR,Arkansas,South,31-AUG-91,1.0,Anti-Black or African American
1,3016,1991,Hope,City,AR,Arkansas,South,19-SEP-91,1.0,Anti-White
2,43,1991,Pine Bluff,City,AR,Arkansas,South,04-JUL-91,1.0,Anti-Black or African American
3,44,1991,Pine Bluff,City,AR,Arkansas,South,24-DEC-91,1.0,Anti-White
4,3017,1991,Pine Bluff,City,AR,Arkansas,South,23-DEC-91,1.0,Anti-White


In [29]:
# Converting all the column heading to lower case 
hate_crimeDF = hate_crimeDF.rename(str.lower,axis='columns')
hate_crimeDF.head()

,incident_id,data_year,pub_agency_name,agency_type_name,state_abbr,state_name,region_name,incident_date,total_individual_victims,bias_desc
0,3015,1991,Rogers,City,AR,Arkansas,South,31-AUG-91,1.0,Anti-Black or African American
1,3016,1991,Hope,City,AR,Arkansas,South,19-SEP-91,1.0,Anti-White
2,43,1991,Pine Bluff,City,AR,Arkansas,South,04-JUL-91,1.0,Anti-Black or African American
3,44,1991,Pine Bluff,City,AR,Arkansas,South,24-DEC-91,1.0,Anti-White
4,3017,1991,Pine Bluff,City,AR,Arkansas,South,23-DEC-91,1.0,Anti-White


In [30]:
# removing years that are not in consideration of our study
hate_crimeDF = hate_crimeDF[hate_crimeDF['data_year'] >= 1993]
print(hate_crimeDF.head())
print(hate_crimeDF.shape)

       incident_id  data_year pub_agency_name agency_type_name state_abbr  \
11256        15761       1993       Anchorage             City         AK   
11257        15762       1993       Anchorage             City         AK   
11258        15763       1993       Anchorage             City         AK   
11259        15764       1993       Anchorage             City         AK   
11260        15765       1993       Anchorage             City         AK   

      state_name region_name incident_date  total_individual_victims  \
11256     Alaska        West     14-FEB-93                       2.0   
11257     Alaska        West     24-MAR-93                       2.0   
11258     Alaska        West     02-JUL-93                       1.0   
11259     Alaska        West     07-JUL-93                       1.0   
11260     Alaska        West     12-JUL-93                       1.0   

                            bias_desc  
11256                 Anti-Gay (Male)  
11257  Anti-Black or Afr

In [31]:
#adding administration column
administration = []
hate_crimeDF.loc[hate_crimeDF['data_year'] <= 1996, 'administration'] = "Clinton 1st Term"
hate_crimeDF.loc[(hate_crimeDF['data_year'] > 1996) & (hate_crimeDF['data_year'] <= 2000), 'administration'] = 'Clinton 2nd Term'
hate_crimeDF.loc[(hate_crimeDF['data_year'] >2000) & (hate_crimeDF['data_year'] <= 2004), 'administration'] = 'W. Bush 1st Term'
hate_crimeDF.loc[(hate_crimeDF['data_year'] > 2004) & (hate_crimeDF['data_year'] <= 2008), 'administration'] = 'W. Bush 2nd Term'
hate_crimeDF.loc[(hate_crimeDF['data_year'] > 2008) & (hate_crimeDF['data_year'] <= 2012), 'administration'] = 'Obama 1st Term'
hate_crimeDF.loc[(hate_crimeDF['data_year'] > 2012) & (hate_crimeDF['data_year'] <= 2016), 'administration'] = 'Obama 2nd Term'
hate_crimeDF.loc[hate_crimeDF['data_year'] >= 2017, 'administration'] = 'Trump 1st Term'
hate_crimeDF

,incident_id,data_year,pub_agency_name,agency_type_name,state_abbr,state_name,region_name,incident_date,total_individual_victims,bias_desc,administration
11256,15761,1993,Anchorage,City,AK,Alaska,West,14-FEB-93,2.0,Anti-Gay (Male),Clinton 1st Term
11257,15762,1993,Anchorage,City,AK,Alaska,West,24-MAR-93,2.0,Anti-Black or African American,Clinton 1st Term
11258,15763,1993,Anchorage,City,AK,Alaska,West,02-JUL-93,1.0,Anti-Black or African American,Clinton 1st Term
11259,15764,1993,Anchorage,City,AK,Alaska,West,07-JUL-93,1.0,Anti-White,Clinton 1st Term
11260,15765,1993,Anchorage,City,AK,Alaska,West,12-JUL-93,1.0,Anti-Asian,Clinton 1st Term
...,...,...,...,...,...,...,...,...,...,...,...
201398,466130,2018,Parkersburg,City,WV,West Virginia,South,14-DEC-18,1.0,Anti-Black or African American,Trump 1st Term
201399,466159,2018,Parkersburg,City,WV,West Virginia,South,14-NOV-18,1.0,Anti-Black or African American,Trump 1st Term
201400,508677,2018,Vienna,City,WV,West Virginia,South,26-NOV-18,1.0,Anti-Asian,Trump 1st Term
201401,463503,2018,State Police:,State Police,WV,West Virginia,South,31-JAN-18,2.0,Anti-White,Trump 1st Term


In [32]:
#saving cleaned data csv
hate_crimeDF.to_csv('cleaned_hate_crime.csv',index = False)

In [45]:
# Create PostgreSQL RDS Database Connection
conn_string = "postgres:{pwd}@database-1.ctvycruujmkn.us-east-2.rds.amazonaws.com"
engine = create_engine(f'postgresql://{conn_string}')

In [46]:
# Check table names
engine.table_names()

OperationalError: (psycopg2.OperationalError) FATAL:  password authentication failed for user "postgres"
FATAL:  password authentication failed for user "postgres"

(Background on this error at: http://sqlalche.me/e/13/e3q8)